In [68]:
import requests 
import os 
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.getenv("API_TOKEN")
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
URL = "https://gorest.co.in/public/v2"
PARAMS = {
    "page": 1}

In [1]:
PAYLOAD = [{"name": "ReNam",
           "email": "renan@github.com",
           "gender": "male",
           "status": "inactive"},

           {"name": "Amanda",
           "email": "amanda@github.com",
           "gender": "female",
           "status": "inactive"},

           {"name": "Teste",
            "email": "teste@github.com",
            "gender": "female",
            "status": "active"},

            {"name": "Teste2",
             "email": "teste2@github.com",
             "gender": "male",
             "status": "inactive"}]

for user in PAYLOAD:
    response = requests.post(f"{URL}/users", json=user, headers=HEADERS)
    response.raise_for_status()
    if response.status_code == 201: 
        print(f"Usuário {user['name']} criado com sucesso!")

NameError: name 'requests' is not defined

In [70]:
response = requests.get(f"{URL}/users", params=PARAMS, headers=HEADERS)
df = pd.DataFrame(response.json())
df.head()

,id,name,email,gender,status
0,8321351,Teste2,teste2@github.com,male,inactive
1,8321350,Teste,teste@github.com,female,active
2,8321349,Amanda,amanda@github.com,female,inactive
3,8321348,ReNam,renan@github.com,male,inactive
4,8115170,Anandamayi Achari,achari_anandamayi@stracke-rowe.test,female,active


In [ ]:
PAYLOAD_PATCH = {"status": "active"}
response = requests.patch(f"{URL}/users/8321349", json=PAYLOAD_PATCH, headers=HEADERS)
response.raise_for_status()
if response.status_code == 200:
    print(f"Usuário atualizado com sucesso!")

Usuário atualizado com sucesso!


In [ ]:
PAYLOAD_PUT = {"name": "Renan",
           "email": "renan@github.com",
           "gender": "male",
           "status": "active"}
response = requests.put(f"{URL}/users/8321348", json=PAYLOAD_PUT, headers=HEADERS)
response.raise_for_status()
if response.status_code == 200: 
    print("Usuário atualizado com sucesso!")

Usuário atualizado com sucesso!


In [ ]:
PAYLOAD_DELETE = ['8321351', '8321350']
for user in PAYLOAD_DELETE: 
    response = requests.delete(f"{URL}/users/{user}", headers=HEADERS)
    response.raise_for_status()
    if response.status_code == 204:
        print("Usuários excluídos com sucesso!")

204
Usuários excluídos com sucesso!
204
Usuários excluídos com sucesso!


In [74]:
response = requests.get(f"{URL}/users", headers=HEADERS, params=PARAMS)
response.raise_for_status()

all_data = response.json()
total_pages = int(response.headers["x-pagination-pages"])

for page in range(2, total_pages + 1):
    PARAMS["page"] = page
    response = requests.get(f"{URL}/users", headers=HEADERS, params=PARAMS)
    response.raise_for_status()
    data = response.json()
    all_data.extend(data)

df = pd.DataFrame.from_dict(all_data, orient="columns")

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1350 non-null   int64 
 1   name    1350 non-null   object
 2   email   1350 non-null   object
 3   gender  1350 non-null   object
 4   status  1350 non-null   object
dtypes: int64(1), object(4)
memory usage: 52.9+ KB


In [76]:
df.isnull().sum()

id        0
name      0
email     0
gender    0
status    0
dtype: int64

In [77]:
df['status'].value_counts()

status
active      1086
inactive     264
Name: count, dtype: int64

In [78]:
df.head()

,id,name,email,gender,status
0,8321348,Renan,renan@github.com,male,active
1,8321349,Amanda,amanda@github.com,female,active
2,8115170,Anandamayi Achari,achari_anandamayi@stracke-rowe.test,female,active
3,8115169,Sanka Namboothiri MD,namboothiri_sanka_md@grady-graham.test,female,active
4,8115168,Prof. Bilwa Chopra,prof_chopra_bilwa@greenfelder.example,female,active


In [84]:
base_marketing = df[df['status'] == "inactive"].copy()
base_marketing["First Name"] = base_marketing['name'].str.split(expand=True)[0]
base_marketing = base_marketing[['First Name', "email"]]
base_marketing.to_csv("dados.csv", index=False)

In [63]:
base_product = df[df['status'] == 'active'].copy()
base_product['gender'].value_counts()

gender
female    619
male      504
Name: count, dtype: int64